# Part 2 - Mapping Yelp Search Results

## Obective

- For this CodeAlong, we will be working with the Yelp API results from last class. 
- You will load in the .csv.gz of your yelp results and prepare the data for visualization.
- You will use Plotly Express to create an interactive map with all of the results.

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- [Advanced Transformations with Pandas - Part 1](https://login.codingdojo.com/m/376/12529/88086)
- [Advanced Transformations with Pandas - Part 2](https://login.codingdojo.com/m/376/12529/88088)

### Goal

- We want to create a map with every restaurant plotted as a scatter plot with detailed information that appears when we hover over a business
- We will use plotly express's `px.scatter_mapbox` function to accomplish this.
    - https://plotly.com/python/scattermapbox/
    
    - We will need a Mapbox API token for some of the options:
        - https://studio.mapbox.com/
    

# Loading Data from Part 1

In [1]:
## Plotly is not included in your dojo-env
!pip install plotly

In [2]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import json

## importing plotly 
import plotly.express as px

In [4]:
## Load in csv.gz
df = pd.read_csv('Data/Seattle-pizza.csv.gz')
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,FVbwpNA1uZEGiM02N4XtUg,blotto-seattle,Blotto,https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...,False,https://www.yelp.com/biz/blotto-seattle?adjust...,40,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 47.61849, 'longitude': -122.31664}",[],"{'address1': '1830 12th Ave', 'address2': '', ...",1.206403e+10,(206) 403-1809,1686.862176,NaN
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media4.fl.yelpcdn.com/bphoto/A08GOZ...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,360,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.5976491915013, 'longitude': -1...",['delivery'],"{'address1': '525 Rainier Ave S', 'address2': ...",1.206330e+10,(206) 329-5133,2166.834011,$$
2,IrohtoYjnAR_vc6w6CRCxA,moto-seattle,Moto,https://s3-media2.fl.yelpcdn.com/bphoto/FBmvjy...,False,https://www.yelp.com/biz/moto-seattle?adjust_c...,136,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.56207, 'longitude': -122.38509}",['delivery'],"{'address1': '4526 42nd Ave SW', 'address2': '...",1.206421e+10,(206) 420-8880,8784.075722,$$
3,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,957,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.615379179632, 'longitude': -12...",['delivery'],"{'address1': '700 E Pine St', 'address2': '', ...",1.206323e+10,(206) 322-6444,2080.359405,$
4,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,339,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 47.6146934, 'longitude': -122.312...","['delivery', 'pickup']","{'address1': '1546 15th Ave', 'address2': '', ...",1.206839e+10,(206) 838-8081,1292.526796,$$


## Required Preprocessing 

- 1. We need to get the latitude and longitude for each business as separate columns.
- We also want to be able to show the restaurants:
    - name,
    - price range
    - address
    - and if they do delivery or takeout.

### Separating Latitude and Longitude

In [18]:
## use .apply pd.Series to convert a dict to columns
df['coordinates'].apply(pd.Series)

,0
0,"{'latitude': 47.61849, 'longitude': -122.31664}"
1,"{'latitude': 47.5976491915013, 'longitude': -1..."
2,"{'latitude': 47.56207, 'longitude': -122.38509}"
3,"{'latitude': 47.615379179632, 'longitude': -12..."
4,"{'latitude': 47.6146934, 'longitude': -122.312..."
...,...
856,"{'latitude': 47.61252119845868, 'longitude': -..."
857,"{'latitude': 47.6688956518864, 'longitude': -1..."
858,"{'latitude': 47.6759, 'longitude': -122.30461}"
859,"{'latitude': 47.676215, 'longitude': -122.205084}"


- Why didn't that work???

Best practice is testing things on a single value in the column instead of applying to the entire column right away unless you are perfectly confident beyond any doubt.

In [19]:
## slice out a single test coordinate
test_coord = df.loc[1, 'coordinates']
test_coord

"{'latitude': 47.5976491915013, 'longitude': -122.313305988361}"

- Its not a dictionary anymore, it starts/ends with double quotes.
    - CSV files can't store iterables (lists, dictionaries) so they get converted to strings.

### Fixing the String-Dictionaries

- The json module has another version of load and dump called `json.loads` and `json.dumps`
    - These are designed to process STRINGS instead of files. 
    
- If we use `json.loads` we can convert our string dictionary into an actual dictionary. 

In [20]:
## Use json.loads on the test coordinate
json.loads(test_coord)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

We got an error when trying to convert the string back into a dictionary
- JSON requires double quotes!

In [10]:
test_coord

"{'latitude': 47.5976491915013, 'longitude': -122.313305988361}"

The outer qutoes are double quotes but the inner quotes for latitude and longitude are single quotes so we must replace them with double quotes.

In [21]:
## replace single ' with " 
test_coord = test_coord.replace("'", '"')
test_coord

'{"latitude": 47.5976491915013, "longitude": -122.313305988361}'

In [22]:
## Use json.loads on the test coordinate, again
json.loads(test_coord)

{'latitude': 47.5976491915013, 'longitude': -122.313305988361}

### Now, how can we apply this same process to the entire column??

In [23]:
## replace ' with " (entire column)
df['coordinates'] = df['coordinates'].str.replace("'", '"')
## apply json.loads
df['coordinates'] = df['coordinates'].apply(json.loads)

In [24]:
## slice out a single test coordinate
df.loc[10,'coordinates']

{'latitude': 47.61304626911818, 'longitude': -122.34035259077139}

Now all the values are dictionaries not strings

### Using Apply with pd.Series to convert a dictionary column into multiple columns

In [25]:
## use .apply pd.Series to convert a dict to columns
df['coordinates'].apply(pd.Series)

,latitude,longitude
0,47.618490,-122.316640
1,47.597649,-122.313306
2,47.562070,-122.385090
3,47.615379,-122.323231
4,47.614693,-122.312764
...,...,...
856,47.612521,-122.201333
857,47.668896,-122.372807
858,47.675900,-122.304610
859,47.676215,-122.205084


In [27]:
## Concatenate the 2 new columns.
df = pd.concat([df, df['coordinates'].apply(pd.Series)], axis = 1)

## Drop the Original Column
df = df.drop(columns='coordinates')

## Check new df
df.head(2)

KeyError: 'coordinates'

## Creating a Simple Map

### Register for MapBox API

Mapbox API: https://www.mapbox.com/

In [28]:
## Load in mapbox api credentials from .secret
with open('/Users/aveld/secret/mapbox_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

- Use the plotly express `set_maptbox_acccess_token` function

In [ ]:
## set mapbox token
mapbox = 

In [ ]:
## use scatter_mapbox for M.V.P map


### Adding Hover Data

- We want to show the restaurants:
    - name
    - price range
    - address
    - and if they do delivery or takeout.
    
    
- We can use the `hover_name` and `hover_data` arguments for `px.scatter_mapbox` to add this info!

In [ ]:
## add hover_name (name) and hover_data for price,rating,location


### Fixing the Location Column

In [ ]:
## slice out a test address
test_addr = df.loc[0, 'location']
test_addr

> Also a string-dictionary...

In [ ]:
## replace ' with "
df['location'] = df['location'].str.replace("'", '"')
df

In [ ]:
## apply json.loads
df['location'] = df['location'].apply(json.loads)
df

> Ruh roh....

- Hmm, let's slice out a test_address again and let's write a function to accomplish this instead.
    - We can use try and except in our function to get around the errors.

### Fixing Addresses - with a custom function


In [ ]:
## slice out test address 
test_addr = df.loc[0, 'location']
test_addr

In [ ]:
## write a function to just run json.loads on the address
def fix_address(test_addr):
    try:
        return json.loads(test_addr)
    except:
        return 'Error'

In [ ]:
## test applying our function
df['location'].apply(fix_address)

- It worked! Now let's save this as a new column (display_location),
and then let's investigate the businesses that had an "ERROR".

In [ ]:
### save a new display_location column using our function


In [ ]:
## filter for businesses with display_location == "ERROR"


In [ ]:
## slice out a new test address and inspect
test_addr = df.loc[437, 'location']
test_addr

> After some more investigation, we would find a few issues with these "ERROR" rows.
1. They contained None.
2. They contained an apostrophe in the name.
3. ...?

### Possible Fixes (if we care to/have the time)


- Use Regular Expressions to find an fix the display addresses with "'" in them
- Use string split to split on the word display address.
    - Then use string methods to clean up

### Moving Forward without those rows (for now)

In [ ]:
## remove any rows where display_location == 'ERROR'


- We want the "display_address" key from the "display_location" dictionaries.
- We could use a .apply and a lamda to slice out the desired key.

In [ ]:
## use apply and lambda to slice correct key


- Almost done! We want to convert display_address to a string instead a list of strings.
- We can use the string method .join to do so!

In [ ]:
## slice out a test_address


In [ ]:
## test using .join with a "\n"


In [ ]:
## apply the join to every row with a lambda


### Final Map

In [ ]:
## make ourn final map and save as varaible


#### HTML Uses `<br>` instead of `\n`

In [ ]:
## remake the final address column with <br> instead 

## plot the final map

In [ ]:
## use fig.write_html to save map
